In [1]:
import subprocess
import zenhan
import os
import sys
import re
import unicodedata
import argparse
import pandas as pd
import pickle
import json
import tqdm

In [2]:
def normalize(text):
    #text = re.sub(r'\d+', '0', text)
    text = re.sub(r'（', '(', text)
    text = re.sub(r'）', ')', text)
    text = re.sub('．','.',text)
    text=re.sub('０','0',text)
    text=re.sub('１','1',text)
    text=re.sub('２','2',text)
    text=re.sub('３','3',text)
    text=re.sub('４','4',text)
    text=re.sub('５','5',text)
    text=re.sub('６','6',text)
    text=re.sub('７','7',text)
    text=re.sub('８','8',text)
    text=re.sub('９','9',text)
    return(text)

In [3]:
#企業リストを取得
company_list = pd.read_csv("../data/new2/df_achieve/company_list.csv",index_col=0)
company_list=company_list.drop(14350)

In [4]:
non_list=[]
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/text_data_sep/'+str(company)
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass

    for code in df["pdf_code"]:
        mokuzi_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'
        text_path =  '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'

        out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
        with open(mokuzi_path, 'rb') as p:
            mokuzi_list=pickle.load(p)
        with open(text_path,'r') as t:
            text = t.read()
        
        text_list=[]
        text_dic={"text":[]}
        #kari_dic={}
        kari_text=text
        true_mokuzi=[]
        false_mokuzi=[]
        try:
            for i,mokuzi in enumerate(mokuzi_list):

                if len(mokuzi)>3:
                    kari=mokuzi[0:3]
                    if "（" in kari or "(" in kari or "（" in kari or kari[0].isdigit() or kari[1].isdigit() or kari[2].isdigit():

                        ind=kari_text.find(mokuzi)
                        if ind!=-1:
                            #目次があったものは変数に入れておく
                            true_mokuzi.append(mokuzi)
                            kari_dic={}
                            if text_dic["text"]!=[]:
                                kari_dic["title"]=true_mokuzi[-2]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-2]):ind]
                                text_dic["text"].append(kari_dic)
                                kari_text=kari_text[ind:]

                                #mokuzi_listの最後に来た時の処理
                                if mokuzi==mokuzi_list[-1]:
                                    kari_dic={}
                                    kari_dic["title"]=true_mokuzi[-1]
                                    kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                    text_dic["text"].append(kari_dic)
                                    break

                            #false_mokuziに目次があるとき（textのない目次）
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]

                            elif i==0:
                                kari_dic["title"]="目次より前"
                                kari_dic["text"]=text[:ind]
                                text_dic["text"].append(kari_dic)
                                kari_text=text[ind:]

                        else:
                            false_mokuzi.append(mokuzi)
                             #mokuzi_listの最後に来た時の処理
                            if mokuzi==mokuzi_list[-1]:
                                kari_dic={}
                                kari_dic["title"]=true_mokuzi[-1]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                text_dic["text"].append(kari_dic)
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]
            #保存
            json_file = open(out_path, 'w')
            json.dump(text_dic, json_file, ensure_ascii=False)
            json_file.close()
        except:
            non_list.append(company)
            

100%|██████████████████████████████████████| 2512/2512 [00:21<00:00, 116.53it/s]


In [5]:
# text=normalize(text)
# kari_mokuzi=[]
# for mokuzi in mokuzi_list:
#     kari_mokuzi.append(normalize(mokuzi))
# mokuzi_list=kari_mokuzi

In [6]:
len(non_list)

322

- 52080
- 95080

In [7]:
kari_list=sorted(list(set(non_list)))

In [8]:
#kari_list=[52080,76150,95080]
#kari_list=[76150,95080]

In [9]:
#エラーの処理
non_list=[]
for company in tqdm.tqdm(kari_list):
    
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/text_data_sep/'+str(company)
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass
    
    for code in df['pdf_code']:
        try:
            mokuzi_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'
            text_path =  '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'

            out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
            with open(mokuzi_path, 'rb') as p:
                mokuzi_list=pickle.load(p)
            with open(text_path,'r') as t:
                text = t.read()

            text=normalize(text)
            text_list=[]
            text_dic={"text":[]}
            #kari_dic={}
            kari_text=text
            true_mokuzi=[]
            false_mokuzi=[]
            for i,mokuzi in enumerate(mokuzi_list):
                mokuzi = mokuzi.replace('\n','')
                mokuzi=normalize(mokuzi)
                mokuzi=mokuzi.replace('..','')
                if mokuzi[-1]==".":
                    mokuzi=mokuzi[:-1]
                if len(mokuzi)>3:
                    kari=mokuzi[0:3]
                    if "（" in kari or "(" in kari or "（" in kari or kari[0].isdigit() or kari[1].isdigit() or kari[2].isdigit():

                        ind=kari_text.find(mokuzi)
                        if ind!=-1:
                            #目次があったものは変数に入れておく
                            true_mokuzi.append(mokuzi)
                            kari_dic={}
                            if text_dic["text"]!=[]:
                                kari_dic["title"]=true_mokuzi[-2]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-2]):ind]
                                text_dic["text"].append(kari_dic)
                                kari_text=kari_text[ind:]

                                #mokuzi_listの最後に来た時の処理
                                if mokuzi==mokuzi_list[-1]:
                                    kari_dic={}
                                    kari_dic["title"]=true_mokuzi[-1]
                                    kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                    text_dic["text"].append(kari_dic)
                                    break

                            #false_mokuziに目次があるとき（textのない目次）
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]

                            elif i==0:
                                kari_dic["title"]="目次より前"
                                kari_dic["text"]=text[:ind]
                                text_dic["text"].append(kari_dic)
                                kari_text=text[ind:]

                        else:
                            false_mokuzi.append(mokuzi)
                             #mokuzi_listの最後に来た時の処理
                            if mokuzi==mokuzi_list[-1]:
                                kari_dic={}
                                kari_dic["title"]=true_mokuzi[-1]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                text_dic["text"].append(kari_dic)
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]
            
             #保存
            json_file = open(out_path, 'w')
            json.dump(text_dic, json_file, ensure_ascii=False)
                                    
        except:
            non_list.append(company)
            break

100%|███████████████████████████████████████████| 48/48 [00:00<00:00, 78.78it/s]


In [10]:
non_list

[36760, 45340, 52080, 63090, 67890, 95080, 97430]

In [11]:
#エラーして残った目次を修正して回したもの
for company in tqdm.tqdm(kari_list):
    
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/text_data_sep/'+str(company)
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass
    
    for code in df['pdf_code']:

        mokuzi_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'
        text_path =  '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'

        out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
        with open(mokuzi_path, 'rb') as p:
            mokuzi_list=pickle.load(p)
        with open(text_path,'r') as t:
            text = t.read()

        text=normalize(text)
        text_list=[]
        text_dic={"text":[]}
        #kari_dic={}
        kari_text=text
        true_mokuzi=[]
        false_mokuzi=[]
        kari_mokuzi=''
        for mokuzi in mokuzi_list:
            kari_mokuzi +=mokuzi +'\n'
    
        new_mokuzi=[]
        for mokuzi in kari_mokuzi.split('\n'):
            if len(mokuzi)>3:
                new_mokuzi.append(mokuzi)

        for i,mokuzi in enumerate(new_mokuzi):
            if mokuzi[0].isdecimal():
                if mokuzi[1]!="．":
                    if mokuzi[2]=="（":
                        new_mokuzi[i]=mokuzi[2:]
                    else:
                         new_mokuzi[i]=mokuzi[1:]
        mokuzi_list=new_mokuzi
            
        for i,mokuzi in enumerate(mokuzi_list):
            mokuzi = mokuzi.replace('\n','')
            mokuzi=normalize(mokuzi)
            mokuzi=mokuzi.replace('..','')
            if mokuzi[-1]==".":
                mokuzi=mokuzi[:-1]
                
            if len(mokuzi)>3:
                kari=mokuzi[0:3]
                if "（" in kari or "(" in kari or "（" in kari or kari[0].isdigit() or kari[1].isdigit() or kari[2].isdigit():

                    ind=kari_text.find(mokuzi)
                    if ind!=-1:
                        #目次があったものは変数に入れておく
                        true_mokuzi.append(mokuzi)
                        kari_dic={}
                        if text_dic["text"]!=[]:
                            kari_dic["title"]=true_mokuzi[-2]
                            kari_dic["text"]=kari_text[len(true_mokuzi[-2]):ind]
                            text_dic["text"].append(kari_dic)
                            kari_text=kari_text[ind:]

                            #mokuzi_listの最後に来た時の処理
                            if mokuzi==mokuzi_list[-1]:
                                kari_dic={}
                                kari_dic["title"]=true_mokuzi[-1]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                text_dic["text"].append(kari_dic)
                                break

                        #false_mokuziに目次があるとき（textのない目次）
                            if false_mokuzi!=[]:
                                for non_mokuzi in false_mokuzi:
                                    kari_dic={}
                                    kari_dic["title"]=non_mokuzi
                                    kari_dic["text"]=""
                                    text_dic["text"].append(kari_dic)
                                false_mokuzi=[]

                        elif i==0:
                            kari_dic["title"]="目次より前"
                            kari_dic["text"]=text[:ind]
                            text_dic["text"].append(kari_dic)
                            kari_text=text[ind:]

                    else:
                        false_mokuzi.append(mokuzi)
                         #mokuzi_listの最後に来た時の処理
                        if mokuzi==mokuzi_list[-1]:
                            kari_dic={}
                            kari_dic["title"]=true_mokuzi[-1]
                            kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                            text_dic["text"].append(kari_dic)
                            if false_mokuzi!=[]:
                                for non_mokuzi in false_mokuzi:
                                    kari_dic={}
                                    kari_dic["title"]=non_mokuzi
                                    kari_dic["text"]=""
                                    text_dic["text"].append(kari_dic)
                                false_mokuzi=[]

         #保存
        json_file = open(out_path, 'w')
        json.dump(text_dic, json_file, ensure_ascii=False)
        json_file.close()

100%|██████████████████████████████████████████| 48/48 [00:00<00:00, 113.70it/s]


In [227]:
mokuzi_list

['2．会計基準の選択に関する基本的な考え方',
 '3．連結財務諸表及び主な注記',
 '(1)連結貸借対照表',
 '連結損益計算書',
 '(追加情報)',
 '(セグメント情報等)',
 '(１株当たり情報)',
 '(重要な後発事象)',
 '4．その他']

In [209]:
kari_mokuzi=''
for mokuzi in mokuzi_list:
    kari_mokuzi +=mokuzi +'\n'
        
new_mokuzi=[]
for mokuzi in kari_mokuzi.split('\n'):
    if len(mokuzi)>3:
        new_mokuzi.append(mokuzi)

for i,mokuzi in enumerate(new_mokuzi):
    if mokuzi[0].isdecimal():
        if mokuzi[1]!="．":
            if mokuzi[2]=="（":
                new_mokuzi[i]=mokuzi[2:]
            else:
                 new_mokuzi[i]=mokuzi[1:]
new_mokuzi=mokuzi_list

In [222]:
new_mokuzi

['\n1．経営成績等の概況',
 '（1）当期の経営成績の概況',
 '2（2）当期の財政状態の概況',
 '（3）当期のキャッシュ・フローの概況',
 '5（4）今後の見通し',
 '2．会計基準の選択に関する基本的な考え方',
 '63．連結財務諸表及び主な注記',
 '（1）連結貸借対照表',
 '7（2）連結損益計算書及び連結包括利益計算書',
 '連結損益計算書',
 '9連結包括利益計算書',
 '（3）連結株主資本等変動計算書',
 '11（4）連結キャッシュ・フロー計算書\n（5）連結財務諸表に関する注記事項',
 '',
 '15（継続企業の前提に関する注記）',
 '（セグメント情報等）',
 '15（1株当たり情報）',
 '（重要な後発事象）']

In [170]:
mokuzi_list

['\n1.当四半期決算に関する定性的情報',
 '(1)経営成績に関する説明',
 '(2)財政状態に関する説明',
 '(3)連結業績予想などの将来予測情報に関する説明',
 '2.四半期連結財務諸表及び主な注記',
 '(1)四半期連結貸借対照表',
 '(2)四半期連結損益計算書及び四半期連結包括利益計算書',
 '四半期連結損益計算書\n第１四半期連結累計期間',
 '四半期連結包括利益計算書\n第１四半期連結累計期間',
 '(3)継続企業の前提に関する注記',
 '(4)株主資本の金額に著しい変動があった場合の注記',
 '(5)四半期連結財務諸表の作成に特有の会計処理の適用',
 '(6)セグメント情報等']

In [123]:
normalize(kari)==false_mokuzi[0]

False

In [50]:
i=1
false_mokuzi[0]

'１.経営成績等の概況'

In [126]:
false_mokuzi[0][0]==kari[0]

False

In [125]:
kari[0]

'1'